In [1]:
import torch
from torch.utils.data import Dataset

import torchaudio
import torchaudio.transforms

import sys, os

from pprint import pprint

from tqdm.autonotebook import tqdm

import json

import numpy as np

import matplotlib.pylab as plt
import seaborn as sns

import librosa
import librosa.display

import pandas as pd

from pathlib import Path

import gc

MANUAL_SEED = 69

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from datetime import date
from datetime import datetime

import os.path
from os import path
  
import json

import time

import copy

from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 150

from sklearn.decomposition import PCA



In [2]:
def make_dir_if_absent(dir_path):
  
  # print("making dir: ", dir_path)
  
  if not os.path.exists(dir_path):
    os.makedirs(dir_path)
    

In [3]:
def dataset_pca(dataset):
  inputs = []
  outputs = []

  dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=1024, num_workers=16, 
  )

  iterator_list = list(iter(dataloader))
  ### len(iterator_list) --> 23
  
  # for batch_input, batch_output in tqdm(iter(dataloader), colour="#FF00FF"):
  for batch_input, batch_output in tqdm(iterator_list[12:], colour="#ff8c00"):
    inputs.append(batch_input) 
    outputs.append(batch_output)

  inputs = torch.cat(tuple(inputs), dim=0).squeeze(1).numpy()
  outputs = torch.cat(tuple(outputs), dim=0).numpy()

  pca = PCA(n_components=2)

  inputs_pca = pca.fit_transform(inputs)

  dataset_pca_tuples = (
    zip(
      inputs_pca[:, 0], inputs_pca[:, 1], outputs, np.argmax(a=outputs, axis=1)
    )
  )

  return pd.DataFrame(
    dataset_pca_tuples, 
    columns = [
      "dataset_pca_1", "dataset_pca_2", "label_one_hot", "label_scalar"
    ]
  )




In [4]:
DATASET_SIZE = "extra_large"
DATASET_FOLDER = "../data/audio"

DATASET_NUM_SAMPLES_PER_SECOND = 8000
DATASET_NUM_CHANNELS = 1

DATASET_NAME = f"fma_{DATASET_SIZE}_organized_by_label_resampled_{DATASET_NUM_SAMPLES_PER_SECOND}_rechanneled_{DATASET_NUM_CHANNELS}"

dataset_path = f"{DATASET_FOLDER}/{DATASET_NAME}"

TRAINING_LOGS_FOLDER = "./logs"

NORMALIZE_AUDIO = True
AUDIO_NUM_FRAMES = 238000

In [5]:
class FMADataset(Dataset):

  def __init__(self, path, normalize_audio, audio_num_frames):
    self.path = path
    self.normalize_audio = normalize_audio
    self.audio_num_frames = audio_num_frames
    
    self.data = self._load_audio_list()

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    
    try: 
      
      waveform, _ = torchaudio.load(
        filepath=self.data[idx], 
        normalize=self.normalize_audio,
        num_frames=self.audio_num_frames
      )
      
      if waveform.shape[1] < self.audio_num_frames:
        waveform = self._apply_padding(waveform)
      
      label = self.data[idx].split("/")[-2]
      label_one_hot = self._label_from_str_to_one_hot(label)
      
      return waveform, label_one_hot
    
    except Exception as e:
      print(f"Got the following exception for the file {self.data[idx]}")
      print("\n\n")
      print(e)
      
  def _apply_padding(self, to_pad):
    padding_size = self.audio_num_frames - to_pad.shape[1]
    
    return torch.nn.functional.pad(
      to_pad, (0, padding_size)
    )
  
  def _label_from_str_to_one_hot(self, label_str: str): 
  
    if label_str == "Pop":
      return torch.tensor([1, 0, 0, 0, 0, 0]).float()
    
    if label_str == "Hip-Hop":
      return torch.tensor([0, 1, 0, 0, 0, 0]).float()
    
    if label_str == "Electronic":
      return torch.tensor([0, 0, 1, 0, 0, 0]).float()
    
    if label_str == "Rock":
      return torch.tensor([0, 0, 0, 1, 0, 0]).float()

    if label_str == "Folk":
      return torch.tensor([0, 0, 0, 0, 1, 0]).float()

    if label_str == "Jazz":
      return torch.tensor([0, 0, 0, 0, 0, 1]).float()
    
  
  def _load_audio_list(self):
    
    audio_path_list = []
    
    for path, subdirs, files in tqdm(os.walk(self.path), colour="magenta"):
      for name in files:
          
        file_audio_path = os.path.join(path, name)
        
        audio_path_list.append(file_audio_path)
        
    return audio_path_list
        
        

In [6]:
# fma_dataset = FMADataset(
fma_dataset = FMADataset(
  path=dataset_path, 
  normalize_audio=NORMALIZE_AUDIO, 
  audio_num_frames=AUDIO_NUM_FRAMES
)

0it [00:00, ?it/s]

In [7]:
dataset_pca_df = dataset_pca(fma_dataset)

make_dir_if_absent(dir_path=f"{dataset_path}_pca")

dataset_pca_df.to_json(
  f"{dataset_path}_pca/{DATASET_NAME}_pca_2.json"
)

# dataset_pca_df.to_csv(
#   f"{dataset_path}_pca/{DATASET_NAME}_pca_1.csv"
# )

  0%|          | 0/11 [00:00<?, ?it/s]